In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')
from src import database as db

# Transactions Classification
The classification uses deterministic approach where the transaction description is matched with the keywords of the account names. The first match is used to assign the debit and credit accounts. If no match is found, the default accounts are used.

## Tokenization of transaction descriptions

In [ ]:
df = pd.read_csv('../data/02_intermediate/01_bank_account/01_bank_account_transactions.csv')
df.info()

In [ ]:
# Tokenize the 'Description' column and create a new column 'Tokens'
def tokenize_text(text):
    return RegexpTokenizer(r'\w+').tokenize(text)
df['tokens'] = df['description'].apply(lambda x: tokenize_text(x))
df.head()

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.update({'value', 'date', 'card', 'au', 'aus', 'xx5824', 'canberra', 'sydney', 'melbourne'})

def remove_tokens(tokens):
    """Removes stopwords and numeric values from a list of tokens"""
    filtered_sentence = []
    for w in tokens:
        if w.lower() not in stop_words and not w.isnumeric():
            filtered_sentence.append(w)
    return filtered_sentence

df['tokens'] = df['tokens'].apply(lambda x: remove_tokens(x))
df.head()

## Get account names and their classification tokens from the database

In [ ]:
conn = db.create_db_connection()
db.add_transaction_category(conn)
categories = db.get_account_names_and_keywords(conn)
conn.close()
print(categories)

## Assign debit and credit accounts to the transactions

In [ ]:
# Add 'Debit' and 'Credit' columns to the dataframe with default values
df.insert(1, 'debit_account', 'Misc')
df.insert(2, 'credit_account', 'Cash')
df.insert(3, 'type', 'Expense')
df.head()

In [ ]:
def assign_category(row, categories):
    """Iterates through the dictionary and assigns debit and credit accounts to the transactions in the dataframe"""
    tokens = row["tokens"]
    amount = row["amount"]
    row['amount'] = abs(amount)
    
    for category, keywords in categories.items():
        for token in tokens:
            if token in keywords:
                if amount < 0:
                    row['debit_account'] = category
                    row['type'] = 'Expense'
                else:
                    row['credit_account'] = category
                    row['type'] = 'Income'
                return row  # Exit the function after assigning the category
    return row # Return the row as is if no category is found

df = df.apply(lambda row: assign_category(row, categories), axis=1)
df.head()

In [ ]:
# Converting datatype of the date column into datetime
df['date']=pd.to_datetime(df['date'],exact=True,format='%d/%m/%Y')

## Analyse the classification results

In [ ]:
df.info()

In [ ]:
df['debit_account'].value_counts()

In [ ]:
# Number of unclassified transactions
len_unclassified = len(df[df['debit_account'] == 'Misc'])
print(len_unclassified)

In [ ]:
# Proportion of unclassified transactions from the total number of transactions
proportion_unclassified = len_unclassified / len(df)
print("The proportion of unclassified transactions is: " + str(proportion_unclassified))

In [ ]:
df['tokens'] = df['tokens'].apply(lambda x: ', '.join(x))

## Save the transactions to a CSV file

In [ ]:
df.to_csv('../data/03_primary/01_bank_account/transactions.csv', index=False)